In [145]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from datetime import datetime
import re


## Configurando driver e url


In [147]:
inicio_script = time.time()


driver = webdriver.Chrome()

url = 'https://www.amazon.com.br/hz/wishlist/ls/2FJRI71SN2K0L?ref_=wl_share' #Coloque aqui o link da sua wishlist

driver.get(url)

if driver.title: #se captar o título da página = sucesso
    print(f'Sucesso ao acessar: {driver.title}')
else:
    print(f'Erro ao acessar.')




Sucesso ao acessar: Amazon.com.br


## Carregando nome da lista de compras

In [149]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

nome_lista = soup.find('span', {'id': 'profile-list-name'})
if nome_lista:
    print(f'Lista: "{nome_lista.text}" carregada com sucesso!')
else:
    print(f'Nome da lista de compras não encontrada')


Lista: "Comprinhas" carregada com sucesso!


## Funções

In [151]:
def extrair_nome_lista(soup):
    nome_lista = soup.find('span', {'id': 'profile-list-name'})
    return nome_lista.text if nome_lista else 'Nome não encontrado'

def obter_data_extracao():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def extrair_itens_lista(soup):
    itens = soup.find_all('li', {'class': 'g-item-sortable'})
    data_extracao = obter_data_extracao()
    return [extrair_dados_itens(item, data_extracao) for item in itens]
    


def extrair_dados_itens(item, data_extracao):
    dados_item = {}
    dados_item['nome'] = item.find('a', {'class': 'a-link-normal'}).get('title', 'Nome não encontrado')
    dados_item['link'] = f"https://www.amazon.com.br{item.find('a', {'class': 'a-link-normal'}).get('href', '')}"
    dados_item['imagem'] = item.find('img').get('src', 'Imagem não encontrada') if item.find('img') else 'Imagem não encontrada'
    dados_item['asin'] = extrair_codigo_asin(dados_item['link'])
    
    elemento_preco = item.find('span', {'class': 'a-price'})
    if elemento_preco:
        preco = elemento_preco.find('span', {'aria-hidden': 'true'}).text.strip()
    else:
        preco = 'Preço indisponível'
    dados_item['preco'] = preco

    dados_item['data_extracao'] = data_extracao
    return dados_item


def extrair_codigo_asin(link):
    pattern = r"/[dg]p/([A-Z0-9]{10})(/|$|\?)"
    match = re.search(pattern, link)

    if match:
        asin = match.group(1).upper()
        return asin
    return None





## Principal

In [153]:

nome_lista = extrair_nome_lista(soup)
print(f'Lista: {nome_lista} - Carregada com sucesso!')


itens_lista = extrair_itens_lista(soup)
if itens_lista:
    print(f'Itens encontrados na lista: {len(itens_lista)}\n')
    for item in itens_lista:
        for chave, valor in item.items():
            print(f'{chave.capitalize()}: {valor}')
        print(f'----------')
    
else:
    print(f'Itens da lista não encontrados')


Lista: Comprinhas - Carregada com sucesso!
Itens encontrados na lista: 9

Nome: Head First Software Architecture: A Learner's Guide to Architectural Thinking
Link: https://www.amazon.com.br/dp/1098134354/?coliid=I3IDR2BYJ3VHXT&colid=2FJRI71SN2K0L&psc=1&ref_=list_c_wl_gv_vv_lig_pi_dp
Imagem: https://m.media-amazon.com/images/I/41bxpv6bl4L._SS220_.jpg
Asin: 1098134354
Preco: R$382,39
Data_extracao: 2025-03-04 15:27:49
----------
Nome: Cabo de rede HP Cat.6 Blindado 2metros Azul DHC-CAT6-FTP - 8WW60AA
Link: https://www.amazon.com.br/dp/B08JZCCZ35/?coliid=I30XMCPHNDLIY2&colid=2FJRI71SN2K0L&psc=0&ref_=list_c_wl_gv_vv_lig_pi_dp
Imagem: https://m.media-amazon.com/images/I/41og2oYjSqL._SS220_.jpg
Asin: B08JZCCZ35
Preco: Preço indisponível
Data_extracao: 2025-03-04 15:27:49
----------
Nome: Mentes Extraordinárias - Pocket: Edição compacta
Link: https://www.amazon.com.br/dp/8550304085/?coliid=I27UY1A33WVL0L&colid=2FJRI71SN2K0L&psc=1&ref_=list_c_wl_gv_vv_lig_pi_dp
Imagem: https://m.media-amazon.c

## Fechar navegador

In [155]:
driver.quit()


fim_script = time.time()
tempo_total_script = fim_script - inicio_script

print(f'Tempo total do script: {tempo_total_script:.2f} segundos')

Tempo total do script: 51.43 segundos
